In [314]:
import pretty_midi
import scipy.misc
import imageio
import os

RANGE_TIME_SHIFT = 100
RANGE_VEL = 32
RANGE_NOTE_ON = 128
RANGE_NOTE_OFF = 128

START_IDX = {
    'note_on' : 0,
    'note_off' : RANGE_NOTE_ON,
    'time_shift': RANGE_NOTE_ON + RANGE_NOTE_OFF,
    'velocity' : RANGE_NOTE_ON + RANGE_NOTE_OFF + RANGE_TIME_SHIFT
}

In [315]:
class SustainAdapter:
    def __init__(self, time, type):
        self.start = time
        self.type = type

In [316]:
class SustainDownManager:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.managed_notes = []
        self._note_dict = {} # key: pitch, value: note.start

    def add_managed_note(self, note: pretty_midi.Note):
        self.managed_notes.append(note)

    def transposition_notes(self):
        for note in reversed(self.managed_notes):
            try:
                note.end = self._note_dict[note.pitch]
            except KeyError:
                note.end = max(self.end, note.end)
            self._note_dict[note.pitch] = note.start

In [317]:
class SplitNote:
    def __init__(self, type, time, value, velocity):
        ## type: note_on, note_off
        self.type = type
        self.time = time
        self.velocity = velocity
        self.value = value

    def __repr__(self):
        return '<[SNote] time: {} type: {}, value: {}, velocity: {}>'.format(self.time, self.type, self.value, self.velocity)

In [318]:
class Event:
    def __init__(self, event_type, value):
        self.type = event_type
        self.value = value

    def __repr__(self):
        return '<Event type: {}, value: {}>'.format(self.type, self.value)

    def to_int(self):
        return START_IDX[self.type] + self.value

    @staticmethod
    def from_int(int_value):
        info = Event._type_check(int_value)
        return Event(info['type'], info['value'])

    @staticmethod
    def _type_check(int_value):
        range_note_on = range(0, RANGE_NOTE_ON)
        range_note_off = range(RANGE_NOTE_ON, RANGE_NOTE_ON+RANGE_NOTE_OFF)
        range_time_shift = range(RANGE_NOTE_ON+RANGE_NOTE_OFF,RANGE_NOTE_ON+RANGE_NOTE_OFF+RANGE_TIME_SHIFT)

        valid_value = int_value

        if int_value in range_note_on:
            return {'type': 'note_on', 'value': valid_value}
        elif int_value in range_note_off:
            valid_value -= RANGE_NOTE_ON
            return {'type': 'note_off', 'value': valid_value}
        elif int_value in range_time_shift:
            valid_value -= (RANGE_NOTE_ON + RANGE_NOTE_OFF)
            return {'type': 'time_shift', 'value': valid_value}
        else:
            valid_value -= (RANGE_NOTE_ON + RANGE_NOTE_OFF + RANGE_TIME_SHIFT)
            return {'type': 'velocity', 'value': valid_value}

In [319]:
def _control_preprocess(ctrl_changes):
    sustains = []

    manager = None
    for ctrl in ctrl_changes:
        if ctrl.value >= 64 and manager is None:
            manager = SustainDownManager(start=ctrl.time, end=None)
        elif ctrl.value < 64 and manager is not None:
            manager.end = ctrl.time
            sustains.append(manager)
            manager = None
        elif ctrl.value < 64 and len(sustains) > 0:
            sustains[-1].end = ctrl.time
    return sustains

In [320]:
def _note_preprocess(susteins, notes):
    note_stream = []

    if susteins:
        for sustain in susteins:
            for note_idx, note in enumerate(notes):
                if note.start < sustain.start:
                    note_stream.append(note)
                elif note.start > sustain.end:
                    notes = notes[note_idx:]
                    sustain.transposition_notes()
                    break
                else:
                    sustain.add_managed_note(note)

        for sustain in susteins:
            note_stream += sustain.managed_notes
    
    else:
        for note_idx, note in enumerate(notes):
            note_stream.append(note)

    note_stream.sort(key= lambda x: x.start)
    return note_stream

In [321]:
def _divide_note(notes):
    result_array = []
    notes.sort(key=lambda x: x.start)

    for note in notes:
        on = SplitNote('note_on', note.start, note.pitch, note.velocity)
        off = SplitNote('note_off', note.end, note.pitch, None)
        result_array += [on, off]
    return result_array

In [322]:
def _make_time_sift_events(prev_time, post_time):
    time_interval = int(round((post_time - prev_time) * 100))
    results = []
    while time_interval >= RANGE_TIME_SHIFT:
        results.append(Event(event_type='time_shift', value=RANGE_TIME_SHIFT-1))
        time_interval -= RANGE_TIME_SHIFT
    if time_interval == 0:
        return results
    else:
        return results + [Event(event_type='time_shift', value=time_interval-1)]


In [323]:
def _snote2events(snote: SplitNote, prev_vel: int):
    result = []
    if snote.velocity is not None:
        modified_velocity = snote.velocity // 4
        if prev_vel != modified_velocity:
            result.append(Event(event_type='velocity', value=modified_velocity))
    result.append(Event(event_type=snote.type, value=snote.value))
    return result

In [324]:
def _event_seq2snote_seq(event_sequence):
    timeline = 0
    velocity = 0
    snote_seq = []

    for event in event_sequence:
        if event.type == 'time_shift':
            timeline += ((event.value+1) / 100)
        if event.type == 'velocity':
            velocity = event.value * 4
        else:
            snote = SplitNote(event.type, timeline, event.value, velocity)
            snote_seq.append(snote)
    return snote_seq

In [325]:
def _merge_note(snote_sequence):
    note_on_dict = {}
    result_array = []

    for snote in snote_sequence:
        # print(note_on_dict)
        if snote.type == 'note_on':
            note_on_dict[snote.value] = snote
        elif snote.type == 'note_off':
            try:
                on = note_on_dict[snote.value]
                off = snote
                if off.time - on.time == 0:
                    continue
                result = pretty_midi.Note(on.velocity, snote.value, on.time, off.time)
                result_array.append(result)
            except:
                print('info removed pitch: {}'.format(snote.value))
    return result_array

In [326]:
def encode_midi(file_path):
    events = []
    notes = []
    mid = pretty_midi.PrettyMIDI(midi_file = file_path)

    # print(mid.instruments)
    for inst in mid.instruments:
        # print(inst)
        # print("=======================")
        # print(inst.notes)
        inst_notes = inst.notes

        # print(inst.control_changes)
        # print("-------------------------")
        # for ctrl in inst.control_changes:
        #     # ctrl.number是延音控制编号
        #     if ctrl.number == 64:
        #         ctrls = _control_preprocess([ctrl])
        #         # if len(ctrls) != 0:
        #         #     print(ctrls)
        #         # print("Done")
        ctrls = _control_preprocess([ctrl for ctrl in inst.control_changes if ctrl.number == 64])
        notes += _note_preprocess(ctrls,inst_notes)
        # print(ctrls)
        # print(notes)
    
    # 分离音符事件
    dnotes = _divide_note(notes)
    dnotes.sort(key=lambda x:x.time)
    # print(dnotes)

    cur_time = 0
    cur_vel = 0
    for snote in dnotes:
        events += _make_time_sift_events(prev_time=cur_time,post_time=snote.time)
        events+= _snote2events(snote=snote,prev_vel=cur_vel)

        cur_time = snote.time
        cur_vel = snote.velocity
    # print(events)

    return [e.to_int() for e in events]

In [327]:
def decode_midi(idx_array,file_path=None):
    event_sequence = [Event.from_int(idx) for idx in idx_array]
    # print(event_sequence)
    snote_seq = _event_seq2snote_seq(event_sequence)
    note_seq = _merge_note(snote_seq)
    note_seq.sort(key=lambda x:x.start)

    mid = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(0, False, "Composed by Super Piano Music Transformer AI.")
    instrument.notes = note_seq

    mid.instruments.append(instrument)
    if file_path is not None:
        mid.write(file_path)
    return mid

In [328]:
if __name__ == "__main__":
    # midi_data = pretty_midi.PrettyMIDI('midi_data_min/demo12.mid')
    # midi_data = pretty_midi.PrettyMIDI('dataset/HappyBirthday.mid')

    # print(midi_data.instruments)

    # image_array = midi_data.instruments[2].get_piano_roll()
    # print(image_array)
    # print(dir(scipy.misc))
    
    # imageio.imsave()
    # imageio.imsave('test.jpg',image_array)

    # encoded = encode_midi('dataset/HappyBirthday.mid')
    # encoded = encode_midi('dataset/爱情恰恰.mid')
    # decode_midi(encoded,file_path='out_put/爱情恰恰.mid')
    # print(encoded)
    # print(encoded)
    
    for filepath,dirnames,filenames in os.walk(r'.\dataset'):
        for filename in filenames:
            try:
                print(os.path.join(filepath,filename))
                encoded = encode_midi(os.path.join(filepath,filename))
                decode_midi(encoded,file_path=os.path.join('.\out_put',filename))
            except Exception as e:
                print(e)

.\dataset\一生痴恋.mid
.\dataset\一见钟情.mid
.\dataset\一言难尽.mid
.\dataset\三大纪律八项注意.mid
.\dataset\不了情.mid
.\dataset\不会哭於你面前.mid
.\dataset\不如甭认识.mid
.\dataset\不留余情.mid
.\dataset\不知不觉想起你.mid
.\dataset\不要对他说.mid
.\dataset\东京爱情故事.mid
.\dataset\东方红.mid
.\dataset\为你打拼.mid
.\dataset\乎你走.mid
.\dataset\二泉映月.mid
.\dataset\亲亲我的宝贝.mid
.\dataset\什么.mid
.\dataset\今夜感受我的爱.mid
.\dataset\今夜我想喝醉.mid
.\dataset\今天夜里总下雨.mid
.\dataset\今宵多珍重.mid
.\dataset\他不爱我.mid
.\dataset\仿如隔世.mid
.\dataset\伤心.mid
.\dataset\伤心酒店.mid
.\dataset\伤痕.mid
.\dataset\似梦迷离.mid
.\dataset\似水年华.mid
.\dataset\似水流年.mid
data byte must be in range 0..127
.\dataset\但愿人长久.mid
.\dataset\何德何能.mid
.\dataset\你令我着迷.mid
.\dataset\你怎麽舍得我难过.mid
.\dataset\你是我的.mid
.\dataset\你是我胸口永远的痛.mid
.\dataset\你知道我的迷惘.mid
.\dataset\倩女幽魂.mid
.\dataset\倩女幽魂2.mid
.\dataset\健康歌.mid
.\dataset\傻女.mid
.\dataset\八月桂花遍地开.mid
.\dataset\共你伤心过.mid
.\dataset\兵歌.mid
.\dataset\其实不想走.mid
.\dataset\其实你心里有没有我.mid
.\dataset\再回首.mid
.\dataset\军港之夜.mid
.\dataset\冲开一切.mid
.\dataset\凭著爱.mid
.